<a href="https://colab.research.google.com/github/shubhamrai1318/Resume_Ranking_System/blob/main/ResumeRankingSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resume Ranking System

## Importing Libraries

In [72]:
import pandas as pd
import re
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords as stp
from sklearn.feature_extraction.text import CountVectorizer
# nltk.download()

## Loading Dataset

In [73]:
# Mounting the Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
df = pd.read_csv('/content/drive/MyDrive/Shubham/CsvFiles/Resume Ranking Data Set.csv')
df

,accomplishments_segment,degree,education_segment,emails,job_titles,links,misc_segment,name,objectives_segment,phone,...,text,university_0,university_1,university_2,university_3,university_4,university_5,url,work_experience,work_segment
0,NaN,"MS , B.TECH , PhD",NaN,sutarvinayak2063@gmail.com,"full stack developer , student","https://github.com/ramsuthar305/SIH2019 , http...",NaN,Prasad Nagar,NaN,+919309161108,...,VINAYAK \nSUTAR \nLinkedIn \n+91 9309161108 ...,"national, mainpuri, uttar pradesh","college of arts, sundargarh, odisha","school of engineering, ernakulam, kerala",NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,NaN,NaN
1,NaN,"B.E , MS , M.TECH",M.Tech. Data Science\nJSS Science And Technolo...,pooj.vs@gmail.com,software engineer,pooj.vs,06/2017 – 07/2019\n07/2018 – 07/2019\nmailto:p...,Software Engineer,NaN,7411786822,...,POOJA V S\nSoftware Engineer\nRich exposure to...,"government, kandhamal, odisha","vidya vardhaka college of engineering, mysore,...",NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,NaN,Merchandising System' using PL/SQL.\nmaintain ...
2,Successfully cleared and completed the course ...,"MS , B.TECH , BS",Degree\nInstitute\nBoard/University\nYear\nPer...,khandelwal.ashwin5@gmail.com,NaN,https://www.linkedin.com/in/ashwin-khandelwal-...,NaN,Ashwin Khandelwal,"I seek challenging environment to work in, tha...",9462710290,...,Ashwin Khandelwal \n Email id – khand...,"narsee monjee institute of management studies,...",NaN,NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,NaN,Organization / Institute\nDescription\nBharat ...
3,NaN,"B.E , MS , BS",NaN,NaN,"work from home , support analyst , software an...",NaN,NaN,Computer Engineering,NaN,919930485336,...,Assessment\nQ1. Why should you be hired for th...,"master, medak, telangana",NaN,NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,1.0,NaN
4,NaN,"MS , B.TECH , BS",NaN,NaN,"work from home , classifier",https://github.com/ajay1013/Titanic-ML-from-Di...,NaN, Python,NaN,918010295832,...,Assessment\nQ1. Why should you be hired for th...,"greater noida institute of technology, gautam ...",NaN,NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,NaN,"B.E , MS , M.TECH",M.Tech (2019-2021) \n Digital Communication\nO...,gauravarora0320@gmail.com,"branch manager , embedded software developer ,...","https://www.linkedin.com/in , https://grv20.gi...",NaN,Gaurav Arora,NaN,+919644527421,...,Gaurav Arora\nFull Stack Developer\ngauravaror...,NaN,NaN,NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,2.0,\nApril 2020 – Present | Remote \n...
1569,NaN,"MS , BS",NaN,hey@ayokunlepaul.dev,"devops , senior android developer , team lead","com.gokada , https://play.google.com/store/app...",NaN,Ayokunle Paul,NaN,9035350600,...,"Ayokunle Paul\nLagos, Nigeria · hey@ayokunlepa...",NaN,NaN,NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,2.0,"Softcom Lagos, Nigeria\nSenior Android Develop..."
1570,NaN,"B.S , M.Sc , M.S , BS , MS",NaN,NaN,"devops engineer , traffic manager , release ma...",https://freesearch.naukri.com/preview/printResume,NaN,DevOps Engineer,NaN,17051313590,...,10/02/2021 https://freesearch.naukri.com/previ...,NaN,NaN,NaN,NaN,NaN,NaN,https://training-resumes.s3.ap-south-1.amazona...,12.0,NaN
1571,NaN,MS,NaN,j.udaysaikiran@gmail.com,classifier,NaN,NaN,Udaya SaiKiran,NaN,+917569799250,...,Udaya SaiKiran.Jonnakuti \n +91-7569799250 ...,"engineering college, ajmer, rajasthan","master, medak, telangana","r.k., shrawasti, uttar pradesh","s.r.k, firozabad, uttar pradesh","harshini degree college, hyderabad, telangana",NaN,https://training-resumes.s3.ap-south-1.amazona...,NaN,NaN


## Handling Null Values

In [75]:
df.isnull().sum()

accomplishments_segment    1507
degree                       38
education_segment          1116
emails                     1509
job_titles                   27
links                       354
misc_segment               1551
name                          1
objectives_segment         1159
phone                       219
projects_segment            893
skills                        0
skills_segment             1105
text                          0
university_0               1007
university_1               1305
university_2               1480
university_3               1545
university_4               1568
university_5               1571
url                           0
work_experience             854
work_segment                893
dtype: int64

## Dropping Unuseful Features

In [76]:
df.drop(['accomplishments_segment','education_segment','emails','misc_segment','name','objectives_segment','phone','projects_segment','skills_segment','text',
         'university_0','university_1','university_2','university_3','university_4','university_5','url','work_segment'],axis=1,inplace=True)
df

,degree,job_titles,links,skills,work_experience
0,"MS , B.TECH , PhD","full stack developer , student","https://github.com/ramsuthar305/SIH2019 , http...","algorithms , interview , visualisation , robot...",NaN
1,"B.E , MS , M.TECH",software engineer,pooj.vs,"core java , algorithms , sql tools , deep lear...",NaN
2,"MS , B.TECH , BS",NaN,https://www.linkedin.com/in/ashwin-khandelwal-...,"digital signal processing , algorithms , capac...",NaN
3,"B.E , MS , BS","work from home , support analyst , software an...",NaN,"big data analytics , award , training , comput...",1.0
4,"MS , B.TECH , BS","work from home , classifier",https://github.com/ajay1013/Titanic-ML-from-Di...,"functionality , algorithms , spark , data visu...",NaN
...,...,...,...,...,...
1568,"B.E , MS , M.TECH","branch manager , embedded software developer ,...","https://www.linkedin.com/in , https://grv20.gi...","digital communication , api , atom , robot , c...",2.0
1569,"MS , BS","devops , senior android developer , team lead","com.gokada , https://play.google.com/store/app...","reporting , functionality , socket.io , mockit...",2.0
1570,"B.S , M.Sc , M.S , BS , MS","devops engineer , traffic manager , release ma...",https://freesearch.naukri.com/preview/printResume,"version control , renewals , nfs , telugu , vi...",12.0
1571,MS,classifier,NaN,"natural language , matplotlib , boost , custom...",NaN


In [77]:
print('After removing the columns from dataset..')
df.isnull().sum()

After removing the columns from dataset..


degree              38
job_titles          27
links              354
skills               0
work_experience    854
dtype: int64

In [78]:
# Fill missing values in 'degree' with the mode
df['degree'].fillna(df['degree'].mode()[0], inplace=True)

# Fill missing values in 'links' with a placeholder ('Missing')
df['links'].fillna('Missing', inplace=True)

# Fill missing values in 'work_experience' with 0
df['work_experience'].fillna(0, inplace=True)

# Fill missing values in 'job_titles' with a placeholder ('No Job')
df['job_titles'].fillna('No Job', inplace=True)


## Data Preprocessing and Transformation

In [79]:

all_degrees = ''
for i in df.degree:
  if len(all_degrees) == 0:
    all_degrees = i
  else:
    all_degrees = all_degrees + ' , ' + i

all_degrees = all_degrees.split(',')
all_degrees = [re.sub("[\s.]","",i).upper() for i in all_degrees]
unique_degrees = set(all_degrees)
unique_degrees

{'BE', 'BS', 'BSC', 'BTECH', 'ME', 'MS', 'MSC', 'MTECH', 'PHD'}

In [80]:
df['bachelor_degrees'] = 'No Degree'
df['master_degrees'] = 'No Degree'
df['docterte_degrees'] = 'No Degree'
df['profiles'] = 'No Profile'

In [81]:

ind = 0
for i in df.degree:
  lst = re.sub("[\s.]","",i).upper( )
  # print(lst)
  for j in lst.split(','):
    if j in ['BE', 'BS', 'BSC', 'BTECH']:
      if df.loc[ind,'bachelor_degrees'] == 'No Degree':
        df.loc[ind,'bachelor_degrees'] = j
      else:
        df.loc[ind,'bachelor_degrees'] = df.loc[ind,'bachelor_degrees'] + ' , '+ j
    elif j in ['ME', 'MS', 'MSC', 'MTECH']:
      if df.loc[ind,'master_degrees'] == 'No Degree':
        df.loc[ind,'master_degrees'] = j
      else:
        df.loc[ind,'master_degrees'] = df.loc[ind,'master_degrees'] + ' , '+ j
    elif j == 'PHD':
      if df.loc[ind,'docterte_degrees'] == 'No Degree':
        df.loc[ind,'docterte_degrees'] = j
      else:
        df.loc[ind,'docterte_degrees'] = df.loc[ind,'docterte_degrees'] + ' , '+ j
  ind +=1

In [82]:
df

,degree,job_titles,links,skills,work_experience,bachelor_degrees,master_degrees,docterte_degrees,profiles
0,"MS , B.TECH , PhD","full stack developer , student","https://github.com/ramsuthar305/SIH2019 , http...","algorithms , interview , visualisation , robot...",0.0,BTECH,MS,PHD,No Profile
1,"B.E , MS , M.TECH",software engineer,pooj.vs,"core java , algorithms , sql tools , deep lear...",0.0,BE,"MS , MTECH",No Degree,No Profile
2,"MS , B.TECH , BS",No Job,https://www.linkedin.com/in/ashwin-khandelwal-...,"digital signal processing , algorithms , capac...",0.0,"BTECH , BS",MS,No Degree,No Profile
3,"B.E , MS , BS","work from home , support analyst , software an...",Missing,"big data analytics , award , training , comput...",1.0,"BE , BS",MS,No Degree,No Profile
4,"MS , B.TECH , BS","work from home , classifier",https://github.com/ajay1013/Titanic-ML-from-Di...,"functionality , algorithms , spark , data visu...",0.0,"BTECH , BS",MS,No Degree,No Profile
...,...,...,...,...,...,...,...,...,...
1568,"B.E , MS , M.TECH","branch manager , embedded software developer ,...","https://www.linkedin.com/in , https://grv20.gi...","digital communication , api , atom , robot , c...",2.0,BE,"MS , MTECH",No Degree,No Profile
1569,"MS , BS","devops , senior android developer , team lead","com.gokada , https://play.google.com/store/app...","reporting , functionality , socket.io , mockit...",2.0,BS,MS,No Degree,No Profile
1570,"B.S , M.Sc , M.S , BS , MS","devops engineer , traffic manager , release ma...",https://freesearch.naukri.com/preview/printResume,"version control , renewals , nfs , telugu , vi...",12.0,"BS , BS","MSC , MS , MS",No Degree,No Profile
1571,MS,classifier,Missing,"natural language , matplotlib , boost , custom...",0.0,No Degree,MS,No Degree,No Profile


In [83]:
ind = 0
for i in df.links:
  lst = re.sub("[\s]","",i)
  # print(lst)
  for j in lst.split(','):
    if j.find('github') != -1:
      df.loc[ind,'profiles'] = 'Github'
    elif j.find('linkedin') != -1:
      if df.loc[ind,'profiles'] == 'No Profile':
        df.loc[ind,'profiles'] = 'Linkedin'
      else:
        df.loc[ind,'profiles'] = df.loc[ind,'profiles'] + ' , ' + 'Linkedin'
  ind +=1

In [84]:
df

,degree,job_titles,links,skills,work_experience,bachelor_degrees,master_degrees,docterte_degrees,profiles
0,"MS , B.TECH , PhD","full stack developer , student","https://github.com/ramsuthar305/SIH2019 , http...","algorithms , interview , visualisation , robot...",0.0,BTECH,MS,PHD,Github
1,"B.E , MS , M.TECH",software engineer,pooj.vs,"core java , algorithms , sql tools , deep lear...",0.0,BE,"MS , MTECH",No Degree,No Profile
2,"MS , B.TECH , BS",No Job,https://www.linkedin.com/in/ashwin-khandelwal-...,"digital signal processing , algorithms , capac...",0.0,"BTECH , BS",MS,No Degree,Linkedin
3,"B.E , MS , BS","work from home , support analyst , software an...",Missing,"big data analytics , award , training , comput...",1.0,"BE , BS",MS,No Degree,No Profile
4,"MS , B.TECH , BS","work from home , classifier",https://github.com/ajay1013/Titanic-ML-from-Di...,"functionality , algorithms , spark , data visu...",0.0,"BTECH , BS",MS,No Degree,Github
...,...,...,...,...,...,...,...,...,...
1568,"B.E , MS , M.TECH","branch manager , embedded software developer ,...","https://www.linkedin.com/in , https://grv20.gi...","digital communication , api , atom , robot , c...",2.0,BE,"MS , MTECH",No Degree,"Github , Linkedin"
1569,"MS , BS","devops , senior android developer , team lead","com.gokada , https://play.google.com/store/app...","reporting , functionality , socket.io , mockit...",2.0,BS,MS,No Degree,No Profile
1570,"B.S , M.Sc , M.S , BS , MS","devops engineer , traffic manager , release ma...",https://freesearch.naukri.com/preview/printResume,"version control , renewals , nfs , telugu , vi...",12.0,"BS , BS","MSC , MS , MS",No Degree,No Profile
1571,MS,classifier,Missing,"natural language , matplotlib , boost , custom...",0.0,No Degree,MS,No Degree,No Profile


## 3. handling the null values of work_experience and converted it's all data into integer format


In [85]:
# list(df['work_experience'].unique())
df.work_experience.value_counts()

 0.0       854
 4.0       151
 2.0        91
 1.0        87
 3.0        81
 5.0        73
 6.0        52
 8.0        34
 7.0        32
 9.0        23
 14.0       14
 11.0       12
 10.0       12
 12.0       11
 13.0        6
 18.0        5
 15.0        4
 17.0        4
 21.0        4
 22.0        2
 2005.0      2
 2004.0      2
 16.0        2
 2003.0      2
 2006.0      2
 20.0        2
 2002.0      1
 2010.0      1
 2001.0      1
 2012.0      1
-3.0         1
 1999.0      1
 2013.0      1
 2008.0      1
 19.0        1
Name: work_experience, dtype: int64

In [86]:

ind = 0
for i in df.work_experience:
  if i < 0:
    df.loc[ind,'work_experience'] = i*-1
  elif i > 40:
    df.loc[ind,'work_experience'] = 0
  ind +=1
df['work_experience'] = df['work_experience'].astype(int)

## Removing the 'degree' and 'links' from the dataset after feature handling

In [87]:
df.drop(['degree','links'],axis=1,inplace=True)


In [88]:
df.isnull().sum()
df

,job_titles,skills,work_experience,bachelor_degrees,master_degrees,docterte_degrees,profiles
0,"full stack developer , student","algorithms , interview , visualisation , robot...",0,BTECH,MS,PHD,Github
1,software engineer,"core java , algorithms , sql tools , deep lear...",0,BE,"MS , MTECH",No Degree,No Profile
2,No Job,"digital signal processing , algorithms , capac...",0,"BTECH , BS",MS,No Degree,Linkedin
3,"work from home , support analyst , software an...","big data analytics , award , training , comput...",1,"BE , BS",MS,No Degree,No Profile
4,"work from home , classifier","functionality , algorithms , spark , data visu...",0,"BTECH , BS",MS,No Degree,Github
...,...,...,...,...,...,...,...
1568,"branch manager , embedded software developer ,...","digital communication , api , atom , robot , c...",2,BE,"MS , MTECH",No Degree,"Github , Linkedin"
1569,"devops , senior android developer , team lead","reporting , functionality , socket.io , mockit...",2,BS,MS,No Degree,No Profile
1570,"devops engineer , traffic manager , release ma...","version control , renewals , nfs , telugu , vi...",12,"BS , BS","MSC , MS , MS",No Degree,No Profile
1571,classifier,"natural language , matplotlib , boost , custom...",0,No Degree,MS,No Degree,No Profile


## Defining the functions to calculate the rank of the resume

In [89]:
lemmatizer = WordNetLemmatizer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in analyzer(doc) if w not in set(stp.words('english')))


In [90]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [91]:
def get_tf_idf_cosine_similarity(job_desc,all_resumes):
    tf_idf_vect = TfidfVectorizer(analyzer=stemmed_words)
    tf_idf_desc_vector = tf_idf_vect.fit_transform([job_desc]).todense()
    tf_idf_resume_vector = tf_idf_vect.transform(all_resumes).todense()
    cosine_similarity_list = []
    for i in range(len(tf_idf_resume_vector)):
        cosine_similarity_list.append(cosine_similarity(tf_idf_desc_vector,tf_idf_resume_vector[i])[0][0])
    return cosine_similarity_list

## Opening the Job Description from the text file

In [92]:
with open('/content/drive/MyDrive/Shubham/CsvFiles/Job Description.txt','r',encoding = 'utf-8') as f:
  file_desc_lst =  [r.replace('\n', '') for r in f.readlines()]